In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/natam-tweets-oct2022/native_tweets_Oct2022.csv


In [2]:
# Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter

# Date and time
import datetime as dt
from datetime import date, datetime, timedelta

# Text, NLP
import spacy
import emot
import contractions
import re
import string

# Visualizations
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets
from tabulate import tabulate

In [3]:
# Import dataset
data = pd.read_csv('../input/natam-tweets-oct2022/native_tweets_Oct2022.csv')

# Check
data.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,search_term
0,1584221836828516352,1584221836828516352,1.666543e+12,2022-10-23 11:34:53,-500,NaN,Activist Sacheen Littlefeather exposed by sist...,en,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,Native American
1,1584221758025973760,1583978820830654465,1.666543e+12,2022-10-23 11:34:35,-500,NaN,@vincescarlet @nasescobar316 The point is you ...,en,[],[],...,NaN,NaN,NaN,NaN,"[{'screen_name': 'vincescarlet', 'name': 'Dani...",NaN,NaN,NaN,NaN,Native American
2,1584221636202729472,1584221636202729472,1.666543e+12,2022-10-23 11:34:06,-500,NaN,https://t.co/GulvGiohqi,zxx,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,Native American
3,1584221631118835712,1584221631118835712,1.666543e+12,2022-10-23 11:34:04,-500,NaN,Complete 4 Page Apa Formatted Essay Native Ame...,en,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,Native American
4,1584221608356388865,1584221608356388865,1.666543e+12,2022-10-23 11:33:59,-500,NaN,"Speaking From Orbit, the First Native-American...",en,[],[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,Native American


# Data Cleaning
- Find and remove columns that are totally empty.
- Find and remove columns that have no variability in responses (all cells have the same value).
- Remove columns that add no value to the analysis.

In [4]:
# Find any columns that are totally empty
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2984 entries, 0 to 2983
Data columns (total 39 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2984 non-null   int64  
 1   conversation_id  2984 non-null   int64  
 2   created_at       2984 non-null   float64
 3   date             2984 non-null   object 
 4   timezone         2984 non-null   int64  
 5   place            5 non-null      object 
 6   tweet            2984 non-null   object 
 7   language         2984 non-null   object 
 8   hashtags         2984 non-null   object 
 9   cashtags         2984 non-null   object 
 10  user_id          2984 non-null   int64  
 11  user_id_str      2984 non-null   int64  
 12  username         2984 non-null   object 
 13  name             2984 non-null   object 
 14  day              2984 non-null   int64  
 15  hour             2984 non-null   int64  
 16  link             2984 non-null   object 
 17  urls          

In [5]:
# List of columns to drop
drop_col_list = []

# Add empty columns to the drop list
for col in data.columns:
    # If the column is totally empty
    if data[col].isnull().sum() == data.shape[0]:
        # Add to drop list
        drop_col_list.append(col)
    else:
        # Do nothing
        continue

# Check
print(f"{len(drop_col_list)} columns have been added to the drop column list.")

10 columns have been added to the drop column list.


In [6]:
# Check if all values are the same in a column
for col in data.columns:
    # Skip if not in drop list
    if col not in drop_col_list:
        # Check if there is no variation 
        if data[col].nunique() == 1:
            # Add to drop list
            drop_col_list.append(col)
        # else
        else:
            # Do nothing
            continue
    # Continue
    else:
        # Do nothing
        continue
        
# Check
print(f"There are {len(drop_col_list)} columns in the drop columns list.")

There are 12 columns in the drop columns list.


Columns that hold no value in the analysis include:
- id
- created_at
- date
- timezone
- user_id
- user_id_str
- link
- search_term

In [7]:
# Print drop column list
drop_col_list

['near',
 'geo',
 'source',
 'user_rt_id',
 'user_rt',
 'retweet_id',
 'retweet_date',
 'translate',
 'trans_src',
 'trans_dest',
 'timezone',
 'retweet']

In [8]:
# Extend drop_col_list
drop_col_list.extend(['id', 'created_at', 'date', 'user_id', 'user_id_str', 'link', 'search_term'])

# Check
print(f"There are {len(drop_col_list)} items in the drop column list.")

There are 19 items in the drop column list.


In [9]:
# Drop columns
df = data.drop(columns=drop_col_list, axis=1)

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2984 entries, 0 to 2983
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   conversation_id  2984 non-null   int64 
 1   place            5 non-null      object
 2   tweet            2984 non-null   object
 3   language         2984 non-null   object
 4   hashtags         2984 non-null   object
 5   cashtags         2984 non-null   object
 6   username         2984 non-null   object
 7   name             2984 non-null   object
 8   day              2984 non-null   int64 
 9   hour             2984 non-null   int64 
 10  urls             2984 non-null   object
 11  photos           2984 non-null   object
 12  video            2984 non-null   int64 
 13  thumbnail        910 non-null    object
 14  nlikes           2984 non-null   int64 
 15  nreplies         2984 non-null   int64 
 16  nretweets        2984 non-null   int64 
 17  quote_url        150 non-null    

## Cleaning Data

Cleaning data will occur in phases. To start, any non-English tweets will be removed.

In [10]:
# Drop if df.language != 'en'
df = df[df['language'] == 'en']

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2540 entries, 0 to 2983
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   conversation_id  2540 non-null   int64 
 1   place            5 non-null      object
 2   tweet            2540 non-null   object
 3   language         2540 non-null   object
 4   hashtags         2540 non-null   object
 5   cashtags         2540 non-null   object
 6   username         2540 non-null   object
 7   name             2540 non-null   object
 8   day              2540 non-null   int64 
 9   hour             2540 non-null   int64 
 10  urls             2540 non-null   object
 11  photos           2540 non-null   object
 12  video            2540 non-null   int64 
 13  thumbnail        578 non-null    object
 14  nlikes           2540 non-null   int64 
 15  nreplies         2540 non-null   int64 
 16  nretweets        2540 non-null   int64 
 17  quote_url        140 non-null    

In [11]:
# Reset index
df.reset_index(inplace=True)

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            2540 non-null   int64 
 1   conversation_id  2540 non-null   int64 
 2   place            5 non-null      object
 3   tweet            2540 non-null   object
 4   language         2540 non-null   object
 5   hashtags         2540 non-null   object
 6   cashtags         2540 non-null   object
 7   username         2540 non-null   object
 8   name             2540 non-null   object
 9   day              2540 non-null   int64 
 10  hour             2540 non-null   int64 
 11  urls             2540 non-null   object
 12  photos           2540 non-null   object
 13  video            2540 non-null   int64 
 14  thumbnail        578 non-null    object
 15  nlikes           2540 non-null   int64 
 16  nreplies         2540 non-null   int64 
 17  nretweets        2540 non-null   

In [12]:
# Drop the language column
df = df.drop(columns='language', axis=1)

# Check
df.head(1)

,index,conversation_id,place,tweet,hashtags,cashtags,username,name,day,hour,urls,photos,video,thumbnail,nlikes,nreplies,nretweets,quote_url,search,reply_to
0,0,1584221836828516352,NaN,Activist Sacheen Littlefeather exposed by sist...,[],[],Bob_Mumford1,Bob Mumford,7,11,['https://www.foxnews.com/entertainment/activi...,[],0,NaN,0,0,0,NaN,Native American,[]


### Indicators and Counts

For certain columns, the presence of an item should be noted. Where multiple items could exist, counts should be attached. These columns include:
- hashtags: count
- cashtags: count
- urls: indicator
- photos: count
- thumbnail: indicator
- quote_url: indicator
- reply_to: count

#### Hashtags

In [13]:
# Iterate through the column 
# Note that this column contains strings, not lists.
def listCounter(column):
    # list
    count_list = []
    for c in column:
        # Set string to manipulate
        c_string = c
        # Replace [ and ] and ''
        c_string = re.sub(r'[\[\]\'\']', '', c_string)
        if len(c_string) > 0:
            # Count hashtags
            count_list.append(len(c_string.split(',')))
        else:
            # Count
            count_list.append(0)
    # Return
    return count_list
    
# Set new column
df['hashtag_counts'] = listCounter(df.hashtags)
    
# Check
df.head(1)

,index,conversation_id,place,tweet,hashtags,cashtags,username,name,day,hour,...,photos,video,thumbnail,nlikes,nreplies,nretweets,quote_url,search,reply_to,hashtag_counts
0,0,1584221836828516352,NaN,Activist Sacheen Littlefeather exposed by sist...,[],[],Bob_Mumford1,Bob Mumford,7,11,...,[],0,NaN,0,0,0,NaN,Native American,[],0


In [14]:
# Add to the hashtag list
def listParser(column):
    # Create list
    parsed_list = []
    for c in column:
        # Set string to manipulate
        c_string = c
        # Replace [ and ] and ''
        c_string = re.sub(r'[\[\]\'\']', '', c_string)
        # If there is a string
        if len(c_string) > 0:
            # Split the remaining string
            for i in c_string.split(','):
                # Append to the list
                parsed_list.append(i)
        # Else
        else:
            # Do nothing
            continue
    # Return parsed list
    return parsed_list

In [15]:
# Create the hashtag list
hashtags = listParser(df.hashtags)

# Print
print(f"There are {len(hashtags)} hashtags found in this dataset.")

There are 1301 hashtags found in this dataset.


In [16]:
# Count of hashtags
hashtag_counter = Counter(hashtags)

# View top 10
hashtag_counter.most_common(10)

[(' tairp', 58),
 ('indigenous', 40),
 (' indigenous', 34),
 (' nativeamerican', 27),
 (' americanindian', 21),
 ('foxnews', 13),
 (' handmadejewelrysale', 13),
 (' handmadejewelry', 13),
 (' southwesternjewelry', 13),
 ('freeleonardpeltier', 13)]

#### Cashtags

In [17]:
# Set a new column 
df['cashtag_counts'] = listCounter(df.cashtags)

# Check
df.head(1)

,index,conversation_id,place,tweet,hashtags,cashtags,username,name,day,hour,...,video,thumbnail,nlikes,nreplies,nretweets,quote_url,search,reply_to,hashtag_counts,cashtag_counts
0,0,1584221836828516352,NaN,Activist Sacheen Littlefeather exposed by sist...,[],[],Bob_Mumford1,Bob Mumford,7,11,...,0,NaN,0,0,0,NaN,Native American,[],0,0


In [18]:
# Create the cashtags list
cashtags = listParser(df.cashtags)

# Print
print(f"There were {len(cashtags)} cashtags in the dataset.")

There were 2 cashtags in the dataset.


#### URL Indicator

This will indicate whether a URL was included as part of the tweet.

In [19]:
# URL indicator list
url_indicator = []

# Iterate through the column
for url in df.urls:
    # URL is present
    if 'http' in url:
        url_indicator.append(1)
    # URL is not present
    else:
        url_indicator.append(0)
        
# Add to the dataframe
df['url_indicator'] = url_indicator

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            2540 non-null   int64 
 1   conversation_id  2540 non-null   int64 
 2   place            5 non-null      object
 3   tweet            2540 non-null   object
 4   hashtags         2540 non-null   object
 5   cashtags         2540 non-null   object
 6   username         2540 non-null   object
 7   name             2540 non-null   object
 8   day              2540 non-null   int64 
 9   hour             2540 non-null   int64 
 10  urls             2540 non-null   object
 11  photos           2540 non-null   object
 12  video            2540 non-null   int64 
 13  thumbnail        578 non-null    object
 14  nlikes           2540 non-null   int64 
 15  nreplies         2540 non-null   int64 
 16  nretweets        2540 non-null   int64 
 17  quote_url        140 non-null    

In [20]:
# Collect URLs
url_list = [u for u in df.urls if 'http' in u]

# Print
print(f"There are {len(url_list)} URLs in the dataset.")

There are 765 URLs in the dataset.


In [21]:
# Check for URLs
url_counter = Counter(url_list)

# Top 10 URLs
url_counter.most_common(10)

[("['https://www.foxnews.com/entertainment/activist-sacheen-littlefeather-exposed-sisters-reported-fraudulent-native-american-identity-lie']",
  39),
 ("['https://www.sfchronicle.com/opinion/openforum/article/Sacheen-Littlefeather-oscar-Native-pretendian-17520648.php']",
  15),
 ("['https://apple.news/AjrF3y_KmSHGOoqdD_d-m1w']", 14),
 ("['https://www.breitbart.com/entertainment/2022/10/23/woke-media-fail-sacheen-littlefeathers-sisters-claim-activist-lied-about-being-native-american/']",
  11),
 ("['https://www.cnn.com/2022/10/18/health/flu-vaccination-inequity/index.html']",
  10),
 ("['https://www.sfchronicle.com/opinion/openforum/article/Sacheen-Littlefeather-oscar-Native-pretendian-17520648.php?utm_campaign=CMS%20Sharing%20Tools%20(Premium)&utm_source=t.co&utm_medium=referral']",
  9),
 ("['https://www.etsy.com/shop/Highway66Treasures?ref=seller-platform-mcnav']",
  8),
 ("['https://www.amazon.com/gp/product/B085VN2SSN/?tag=hunchpress-20/']", 7),
 ("['https://thehornnews.com/biden-q

#### Photos

In [22]:
# Photo indicator list
photo_indicator = []

# Iterate through the column
for p in df.photos:
    # If a photo is attached
    if 'http' in p:
        photo_indicator.append(1)
    # If no photo attached
    else:
        photo_indicator.append(0)
        
# Add to the dataframe
df['photo_indicator'] = photo_indicator

In [23]:
# Drop the photo column from the dataframe
df = df.drop(columns='photos', axis=1)

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            2540 non-null   int64 
 1   conversation_id  2540 non-null   int64 
 2   place            5 non-null      object
 3   tweet            2540 non-null   object
 4   hashtags         2540 non-null   object
 5   cashtags         2540 non-null   object
 6   username         2540 non-null   object
 7   name             2540 non-null   object
 8   day              2540 non-null   int64 
 9   hour             2540 non-null   int64 
 10  urls             2540 non-null   object
 11  video            2540 non-null   int64 
 12  thumbnail        578 non-null    object
 13  nlikes           2540 non-null   int64 
 14  nreplies         2540 non-null   int64 
 15  nretweets        2540 non-null   int64 
 16  quote_url        140 non-null    object
 17  search           2540 non-null   

#### Thumbnail

In [24]:
# Thumbnail indicator list
thumbnail_indicator = []

# Iterate through the column
for t in df.thumbnail:
    # If there is no thumbnail
    if isinstance(t, float):
        thumbnail_indicator.append(0)
    # There is an indicator
    else:
        thumbnail_indicator.append(1)
        
# Add to the dataframe
df['thumbnail_indicator'] = thumbnail_indicator

In [25]:
# Drop the thumbnail column
df = df.drop(columns='thumbnail', axis=1)

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                2540 non-null   int64 
 1   conversation_id      2540 non-null   int64 
 2   place                5 non-null      object
 3   tweet                2540 non-null   object
 4   hashtags             2540 non-null   object
 5   cashtags             2540 non-null   object
 6   username             2540 non-null   object
 7   name                 2540 non-null   object
 8   day                  2540 non-null   int64 
 9   hour                 2540 non-null   int64 
 10  urls                 2540 non-null   object
 11  video                2540 non-null   int64 
 12  nlikes               2540 non-null   int64 
 13  nreplies             2540 non-null   int64 
 14  nretweets            2540 non-null   int64 
 15  quote_url            140 non-null    object
 16  search

#### Quote URL

In [27]:
# Quote URL indicator
qurl_indicator = []

# Iterate through the column
for q in df.quote_url:
    # No quote url
    if isinstance(q, float):
        qurl_indicator.append(0)
    # Quote URL exists
    else:
        qurl_indicator.append(1)
        
# Add to dataframe
df['qurl_indicator'] = qurl_indicator

In [28]:
# Drop the quote url column
df = df.drop(columns='quote_url', axis=1)

# Check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540 entries, 0 to 2539
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                2540 non-null   int64 
 1   conversation_id      2540 non-null   int64 
 2   place                5 non-null      object
 3   tweet                2540 non-null   object
 4   hashtags             2540 non-null   object
 5   cashtags             2540 non-null   object
 6   username             2540 non-null   object
 7   name                 2540 non-null   object
 8   day                  2540 non-null   int64 
 9   hour                 2540 non-null   int64 
 10  urls                 2540 non-null   object
 11  video                2540 non-null   int64 
 12  nlikes               2540 non-null   int64 
 13  nreplies             2540 non-null   int64 
 14  nretweets            2540 non-null   int64 
 15  search               2540 non-null   object
 16  reply_

In [ ]:
# 